### Import Libraries

In [1]:

import pandas as pd
import numpy as np

from sklearn import linear_model, model_selection, preprocessing, metrics, svm
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier

import nltk
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
import string
# nltk.download('stopwords')
# nltk.download('punkt')

from plotnine import *

# tensorflow and Keras
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

# module_url = "https://tfhub.dev/google/nnlm-en-dim128/2"
# embed = hub.KerasLayer(module_url)
# embeddings = embed(["A long sentence.", "single-word",
#                   "http://example.com"])
# print(embeddings.shape)  #(3,128)

### Read Data

In [2]:
# read data
train = pd.read_csv( '../data/nlp-getting-started/train.csv' )
test  = pd.read_csv( '../data/nlp-getting-started/test.csv' )
subm_samp = pd.read_csv( '../data/nlp-getting-started/sample_submission.csv' )

train.head(5)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [3]:
##
## Checking size of each class on train set
print( train[ train['target'] == 1 ].shape[0] )
print( train[ train['target'] == 0 ].shape[0] )

3271
4342


### Feature Engineering

In [4]:
###
### Function to process data
###
def textProcessing(dt, textCol):
    
    # make copy
    dt_copy = dt.copy()
    
    # Get stopwords
    stop_words = set(stopwords.words('english')) 
    
    # Remove cases
    dt_copy['tmp_text'] = dt[textCol].str.lower()

    # Remove punctuation
    dt_copy['tmp_text'] = dt_copy.apply(lambda row: row['tmp_text'].translate(str.maketrans('', '', string.punctuation)), axis=1)

    # Tokenize
    dt_copy['tmp_text'] = dt_copy.apply(lambda row: word_tokenize(row['tmp_text']), axis=1)

    # Remove stopwords
    dt_copy['new_text'] = dt_copy['tmp_text'].apply(lambda x: [item for item in x if item not in stop_words])
    
    # Return dataframe
    return dt_copy
    

In [5]:
##
## Load pre-trained sentence embedding - Google's Swivel
model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(model, output_shape=[20], input_shape=[], 
#                            dtype=tf.string, trainable=False)
                           dtype=tf.string, trainable=True)

In [6]:
##
## Define model's architecture
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [8]:
X_train_all = train['text']
Y_train_all = train['target']

# Separate in test/train sets
X_train, X_test, y_train, y_test = train_test_split( X_train_all, Y_train_all, test_size=0.3, random_state=710)

In [9]:
## Train Model
model.fit(X_train,
          y_train,
          epochs=10,
          validation_data=(X_test, y_test),
          verbose=1)

Train on 5329 samples, validate on 2284 samples
Epoch 1/10
5329/5329 [==============================] - 1s 210us/sample - loss: 0.6415 - accuracy: 0.6256 - val_loss: 0.5415 - val_accuracy: 0.7399
Epoch 2/10
5329/5329 [==============================] - 1s 117us/sample - loss: 0.5196 - accuracy: 0.7544 - val_loss: 0.5031 - val_accuracy: 0.7697
Epoch 3/10
5329/5329 [==============================] - 1s 117us/sample - loss: 0.4760 - accuracy: 0.7838 - val_loss: 0.4876 - val_accuracy: 0.7780
Epoch 4/10
5329/5329 [==============================] - 1s 117us/sample - loss: 0.4395 - accuracy: 0.8069 - val_loss: 0.4778 - val_accuracy: 0.7885
Epoch 5/10
5329/5329 [==============================] - 1s 114us/sample - loss: 0.4057 - accuracy: 0.8249 - val_loss: 0.4731 - val_accuracy: 0.7903
Epoch 6/10
5329/5329 [==============================] - 1s 116us/sample - loss: 0.3709 - accuracy: 0.8420 - val_loss: 0.4720 - val_accuracy: 0.7850
Epoch 7/10
5329/5329 [==============================] - 1s 119us

In [10]:
# get predictions for out of sample
predictions = model.predict(X_test)

# get metrics
results = model.evaluate(X_test, y_test)
print(results)

2284/2284 [==============================] - 0s 55us/sample - loss: 0.5172 - accuracy: 0.7750
[0.5172323067321878, 0.7749562]


In [11]:
def recall(y_true, y_pred):
    true_positives = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)))
    possible_positives = np.sum(np.round(np.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives)
    return recall

def precision(y_true, y_pred):
    true_positives = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)))
    predicted_positives = np.sum(np.round(np.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives)
    return precision

prec = precision( np.reshape(y_test.to_numpy(), (-1, 1)), np.round(predictions) )
reca = recall( np.reshape(y_test.to_numpy(), (-1, 1)), np.round(predictions) )
f1   = 2*((prec*reca)/(prec+reca))
print(prec)
print(reca)
f1

0.7281153450051493
0.7387669801462905


0.7334024896265561

### Tf-idf + Bernoulli Naive Bayes Classifier

In [ ]:


##
## Count words
count_vect = CountVectorizer()
train_counts = count_vect.fit_transform(train.text)
train_counts.shape


##
## TF-IDF
tfidf_transformer = TfidfTransformer()
train_tfidf = tfidf_transformer.fit_transform(train_counts)
train_tfidf.shape

##
## Fit Bernoulli Naive-Bayes
clf_tfidf = BernoulliNB().fit(train_tfidf, train.target)
clf_tfidf


##
## Predict on train set
nObs = train_tfidf.shape[0]
print( np.mean( clf_tfidf.predict(train_tfidf[0:nObs]) == train.target[0:nObs] ) )
metrics.f1_score( y_true = clf_tfidf.predict(train_tfidf[0:nObs]),
                  y_pred = train.target[0:nObs] )